# effective core

    omega: the initial co-ownership structure
        P: the preference matrix
           larger number represents higher utility
           

# Notifications

    we have more improved version of assign function

In [124]:
from itertools import combinations
import csv

def output(A):
    '''
    输出矩阵
    '''
    for i in range(len(A)):
        for j in range(len(A[i])):
            print(A[i][j], end = '    ')
        print('\n')
    for i in range((len(A[0])-1)*5+1):
        print('-', end = '')
    print('\n')
    return None

def inpt(file):
    '''
    file is the name of the inputed csv file
    '''
    C = []
    # 从文件中读取数据
    with open(file) as c_file:
        c_reader = csv.reader(c_file, delimiter = ' ')
        for row in c_reader:
            C.append(row)

    for i in range(len(C)):
        for j in range(len(C[i])):
            C[i][j] = int(C[i][j])
    
    return C


def assign(i, S):
    '''
    输入 assignment S 和个体，返回在该分配下，他所分配的物品
    如果一个人分配到了多个物品，返回效用最高的那个
    返回零代表该分配下他什么都没得到
    '''
    if not i in I:
        print('player is out of range')
        return None
    A = []
    for a in O:
        if S[i-1][a-1]:
            A.append(a)
    if not len(A):
        return 0
    elif len(A)==1:
        return A[0]
    else:
        t = A[0]
        for a in A:
            if P_c(i, a, t):
                t = a
        return t


def R_c(i, a, b):
    '''
    判断对i来说，是否偏好a
    '''
    if a>0:
        ua = P[i-1][a-1]
    else:
        ua = 0
    if b>0:
        ub = P[i-1][b-1]
    else:
        ub = 0
        
    if ua>=ub:
        return True
    else:
        return False


def P_c(i, a, b):
    '''
    判断对i来说，是否严格偏好a
    '''
    if a>0:
        ua = P[i-1][a-1]
    else:
        ua = 0
    if b>0:
        ub = P[i-1][b-1]
    else:
        ub = 0
        
    if ua>ub:
        return True
    else:
        return False

    
def private_own(coalition, Omega):
    '''
    输入所有权结构/分配
    返回物品集（在该所有权结构/分配下被该集团完全所有的物品）
    '''
    A = []
    for a in O:
        k = 1
        for i in I:
            if Omega[i-1][a-1] and not (i in coalition):
                k = 0
                break
        if k:
            A.append(a)
    return A


def transpose(A):
    '''
    transpose a matrix
    '''
    n = len(A)
    m = len(A[0])
    ans = empty(m, n)
    for i in range(m):
        for j in range(n):
            ans[i][j] = A[j][i]
    return ans


def empty(n, m):
    '''
    创建 n*m 的零矩阵
    '''
    A = []
    for i in range (n):
        A.append([])
        for j in range(m):
            A[i].append(0)
    return A

In [125]:
# 基础信息的输入
C = inpt('1_C.txt')
output(C)
P = inpt('1_P.txt')
#output(P)


# 派生信息的计算
n_player = len(C)
I = []
for i in range(n_player):
    I.append(i+1)
n_object = len(C[0])
O = []
for a in range(n_object):
    O.append(a+1)


# 其它分配方式
mu1 = inpt('1_mu1.txt')
mu2 = inpt('1_mu2.txt')
sigma = inpt('1_sigma.txt')

1    1    

0    0    

0    0    

------



In [131]:
# 当考虑分配的时候，是不存在共有问题的
# 只有禀赋权才会有共有问题存在
  
def effectively_block(coalition, mu, sigma):
    '''
    coalition effectively blocks mu via sigma
    '''
    
    # 集团中所有人都不变差
    k1 = 1
    for i in coalition:
        if not R_c(i, assign(i, sigma), assign(i, mu)):
            k1 = 0
    
    # 集团中有人严格变好
    k2 = 0
    for i in coalition:
        if P_c(i, assign(i, sigma), assign(i, mu)):
            k2 = 1
    
    # 进行的是集团内部分配
    k3 = 0
    if set(private_own(coalition, sigma))<=set(private_own(coalition, C)):
        k3 = 1
        
    # 新要求
    k4 = 1
    # 这里要加入新的判定条件
    for popu in range(len(coalition)):
        for coal in combinations(coalition, popu+1):
            coal = list(coal)
            if set(private_own(coal, sigma))<=set(private_own(coal, C)):
                obj = set(private_own(coal, C))-set(private_own(coal, sigma))
                if len(obj):
                    for a in obj:
                        if a in private_own(coalition, sigma):
                            k4 = 0
    if coalition == I:
        k4 = 1
    
    if k1 and k2 and k3 and k4:
        return True
    else:
        return False
    
    
def if_in_effective_core(mu, detail = False):
    
    # check if the strong core is empty
    # 对coalition进行组合
    COALITION = []

    for popu in range(1, n_player):
        for coalition in combinations(I, popu+1):
            COALITION.append(list(coalition))
            
    ans = True
    I_plus = [0]+I
    for i in I_plus:
        T1 = []
        for j in I:
            if j==i:
                T1.append(1)
            else:
                T1.append(0)
        for k in I_plus:
            T2 = []
            for m in I:
                if m==k:
                    T2.append(1)
                else:
                    T2.append(0)
            T = transpose([T1, T2])
            for coalition in COALITION:
                if effectively_block(coalition, mu, T):
                    if detail:
                        print('Coalition: '+str(coalition)+'\n')
                        output(T)
                    ans = False
    return ans

In [132]:
# 暂时和 weakly block 完全相同
effectively_block([1,2,3], mu1, sigma)

False

In [134]:
if_in_effective_core(mu2, detail=True)

Coalition: [1, 2, 3]

1    0    

0    1    

0    0    

------

Coalition: [1, 2, 3]

1    0    

0    0    

0    1    

------



False